In [1]:
import os
from dataclasses import dataclass, field
from pathlib import Path

from omegaconf import OmegaConf, ValidationError

from pdf_rag.react_agent_multi_pdfs import ReActAgentMultiPdfs
from dotenv import load_dotenv

load_dotenv()


@dataclass
class ReActAgentConfig:
    data_dir: Path | str
    api_key_gemini: str | None = None
    api_key_mistral: str | None = None
    num_workers: int = 16
    chunks_top_k: int = 5
    nodes_top_k: int = 10
    max_iterations: int = 20
    verbose: bool = True
    queries: list[str] = field(default_factory=list)

    def __post_init__(self):
        self.data_dir = Path(self.data_dir)
        self.root_dir = self.data_dir / "pdfs"
        self.pdfs_dir = self.data_dir / "pdfs"
        self.cache_dir = self.data_dir / "cache"
        self.storage_dir = self.data_dir / "storage_queries"

        self.api_key_gemini = self.api_key_gemini or os.environ.get("GEMINI_API_KEY")
        self.api_key_mistral = self.api_key_mistral or os.environ.get("MISTRAL_API_KEY")
        if not self.api_key_gemini:
            raise ValueError(
                "Gemini API Key is required. Provide api_key_gemini or set GEMINI_API_KEY environment variable."
            )
        if not self.api_key_mistral:
            raise ValueError(
                "Mistral API Key is required. Provide api_key_mistral or set MISTRAL_API_KEY environment variable."
            )


def load_and_validate_config(config_path: str) -> ReActAgentConfig:
    try:
        config = OmegaConf.load(config_path)
        react_agent_schema = OmegaConf.structured(ReActAgentConfig)
        react_agent_config = OmegaConf.merge(react_agent_schema, config)
        react_agent_config = ReActAgentConfig(**react_agent_config)
        print("Configuration loaded and validated successfully:")
        return react_agent_config
    except ValidationError as e:
        raise ValidationError(f"Validation error: {e}")
    except Exception as e:
        raise Exception(f"Error loading configuration: {e}")

/home/nicolas/PycharmProjects/pdf-rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_path = "../configs/query_multi_pdfs_nicolas.yaml"
config = load_and_validate_config(config_path)

react_agent_multi_pdfs = ReActAgentMultiPdfs(
    api_key_gemini=config.api_key_gemini,
    api_key_mistral=config.api_key_mistral,
    root_dir=config.root_dir,
    pdfs_dir=config.pdfs_dir,
    cache_dir=config.cache_dir,
    storage_dir=config.storage_dir,
    num_workers=config.num_workers,
    chunks_top_k=config.chunks_top_k,
    nodes_top_k=config.nodes_top_k,
    max_iterations=config.max_iterations,
    verbose=config.verbose,
)

responses = react_agent_multi_pdfs.process_queries(queries=config.queries)
for q, r in zip(config.queries, responses):
    print(30 * "-")
    print(f"Query: {q}")
    print(f"Response: {r}")

Configuration loaded and validated successfully:


Building agents: 100%|██████████| 6/6 [00:00<00:00, 14.39it/s]


> Running step 0dc45634-70c7-4656-9b12-fc2b086848ce. Step input: What are the vulnerabilities introduced by relying on application programming interfaces (APIs) in Banking as a Service (BaaS)?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: tool_deloitte-tech-risk-sector-banking
Action Input: {'input': 'vulnerabilities of APIs in Banking as a Service (BaaS)'}
> Running step b574597e-591e-42f5-b2fe-075603369f1c. Step input: vulnerabilities of APIs in Banking as a Service (BaaS)
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_deloitte-tech-risk-sector-banking
Action Input: {'input': 'API vulnerabilities in Banking as a Service (BaaS)'}
Observation: BaaS increasingly relies on application programming interfaces, introducing vulnerabilities that can pose risks for banks, such as customers' sensitive data and funds being at risk from phishing an